In [1]:
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer

def PreProcessing():
    df = pd.read_csv("comments.csv", encoding = "utf-8")
    
    df["text"] = df["full_text"]
    df["text"] = df["text"].str.lower()
    df["text"] = [re.sub(r"(?:\@|https?\://)\S+", "", text) for text in df["text"]]
    
    intab = "áéíóúïü"
    outtab = "aeiouiu"
    trantab = str.maketrans(intab, outtab)
    df["text"] = [text.translate(trantab) for text in df["text"]]
    
    df["text"] = df["text"].str.replace("[^a-zñ\s]", "")
    
    st = SnowballStemmer("spanish")
    df["text"] = df["text"].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    
    return df

In [2]:
import pandas as pd
import warnings
import numpy as np
from gensim.models import word2vec

size = 100
w2v_model = None

def GetFVs(comment):
    words = str(comment).split()
    vector = np.zeros(size)
    k = 0
    for w in words:
        if w in w2v_model.wv.vocab:
            vector = vector + w2v_model[w].tolist()
            k = k + 1
    
    if k > 0:
        mean_vector = vector / k
    else:
        mean_vector = vector
            
    return mean_vector

def FeatureExtraction(df):
    warnings.filterwarnings("ignore")
    global w2v_model
    
    df2 = pd.DataFrame(df["text"].values)
    np.savetxt("comments.txt", df2, fmt = "%s", encoding = "utf-8")
    comments = word2vec.Text8Corpus("comments.txt")
    w2v_model = word2vec.Word2Vec(sentences = comments, size = size, min_count = 3)
    y = df["class"]
    
    comments_fvs = []
    for comment in df["text"]:
        fvs = GetFVs(comment)
        comments_fvs.append(fvs)

    column_names = ["f" + str(i) for i in range(1, size + 1)]
    column_names.append("class")

    x = comments_fvs
    df = pd.DataFrame(x) 
    df["class"] = y
    df.columns = column_names

    return df

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score

scaler = None

def MachineLearning(df):
    df.drop(df[ df["class"] == "neutro"].index, inplace = True)
    
    x = df.drop("class", axis = 1, inplace = False)
    df["class"] = [5 if tag == "positivo"  else 1 for tag in df["class"]]
    y = df["class"]
    
    global scaler
    
    scaler = StandardScaler().fit(x)
    normalized_x = scaler.transform(x)
    x_train, x_test, y_train, y_test = train_test_split(normalized_x,
                                                        y,
                                                        test_size = 0.2,
                                                        random_state = 0,
                                                        stratify = y)

    param_grid = {  
        'C': [100],
        'gamma': [0.001],
        'kernel': ['rbf'],
        'tol': [0.001]
    }
    
    grid_search = GridSearchCV(estimator = SVC(),
                               param_grid = param_grid,
                               scoring = 'f1',
                               cv = 5)

    grid_search.fit(x_train, y_train)
    support_vector_model = grid_search
    support_vector_model_train_prediction = support_vector_model.predict(x_train)
    support_vector_model_test_prediction = support_vector_model.predict(x_test)
    
    return support_vector_model

In [4]:
def PreProcess(comment):
    comment = comment.lower()
    comment = re.sub(r"(?:\@|https?\://)\S+", "", comment) 
    
    intab = "áéíóúïü"
    outtab = "aeiouiu"
    trantab = str.maketrans(intab, outtab)
    comment = comment.translate(trantab)
    
    comment = comment.replace("[^a-zñ\s]", "")
    
    st = SnowballStemmer("spanish")
    comment = " ".join([st.stem(word) for word in comment.split()])
    
    return comment

In [5]:
def Predict(model, comment):
    comment = PreProcess(comment)
    x = GetFVs(comment).reshape(1, -1)
    x = scaler.transform(x)
    predicted = "positivo" if model.predict(x)[0] == 5 else "negativo"
    
    return predicted

In [6]:
def TrainModel():
    df = PreProcessing()
    df = FeatureExtraction(df)
    model = MachineLearning(df)
    
    return model

In [7]:
model = TrainModel()

In [12]:
comment = "Muy buena labor del congresita, exitos!"
print(Predict(model, comment))

positivo


In [13]:
comment = "Congresista, corrupto, es un vendido. Deberia estar en la carcel"
print(Predict(model, comment))

negativo


In [14]:
comment = "Muchas gracias por el buen trabajo, espero verlo en el segundo periodo"
print(Predict(model, comment))

positivo


In [15]:
comment = "El robo mucho dinero al peru, no tiene sangre en la cara"
print(Predict(model, comment))

negativo


In [1]:
import pandas as pd
df = pd.read_csv("comments.csv", encoding = "utf-8")


In [2]:
df

,full_text,class
0,totalmente satisfecho x q resolvieron mi cons...,positivo
1,la atencion fue rapida y clara,positivo
2,me atendieron bien,positivo
3,fue muy rapido la atencion,positivo
4,por que me atendieron muy amablemente con buen...,positivo
...,...,...
13285,hice un abono de la cual no me comunico que p...,neutro
13286,atencion,neutro
13287,la demora en la atencion en ventanilla,negativo
13288,que atiendan en todas las cajas para que el mo...,neutro
